In [127]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
from sklearn import preprocessing
import statistics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, log_loss, average_precision_score
from random import sample, randint
from copy import deepcopy
from sklearn.model_selection import cross_validate

In [109]:
from contextlib import contextmanager
from timeit import default_timer

@contextmanager
def elapsed_timer():
    start_time = default_timer()

    class _Timer():
      start = start_time
      end = default_timer()
      duration = end - start

    yield _Timer

    end_time = default_timer()
    _Timer.end = end_time
    _Timer.duration = end_time - start_time

In [2]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [45]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(newsgroups_train.data)
y_train = newsgroups_train.target
x_test = vectorizer.transform(newsgroups_test.data)
y_test = newsgroups_test.target
classes_count = 20

Change dimensions to 500

In [46]:
svd = TruncatedSVD(n_components=500)
svd.fit(x_train)
svd_x_train = svd.transform(x_train)
svd_x_test = svd.transform(x_test)

In [5]:
#x_train_res, y_train_res = resample(svd_x_train, y_train, n_samples=5000, replace=False, random_state=0)
#x_test_res, y_test_res = resample(svd_x_test, y_test, n_samples=500, replace=False, random_state=0)
#x_train_res = x_train_res.reshape((x_train_res.shape[0],-1))
#x_test_res = x_test_res.reshape((x_test_res.shape[0],-1))
#y_train_res = y_train_res.reshape((y_train_res.shape[0],))

In [6]:
#scaler = StandardScaler(with_mean=False).fit(x_train)
#x_train_scaled = scaler.transform(x_train) # scaling data
#x_test_scaled = scaler.transform(x_test)
#print(x_train_scaled.shape)

SVM on full data and full features

In [47]:
clf = SVC(probability=True)
clf.fit(svd_x_train, y_train)
#clf.fit(x_train_res, y_train_res)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [48]:
y_pred = clf.predict(svd_x_test)
#y_pred = clf.predict(x_test_res)

In [49]:
accuracy = accuracy_score(y_test, y_pred)
#accuracy = accuracy_score(y_test_res, y_pred)

In [50]:
accuracy #0.78956

0.7895645246946362

In [51]:
f_score = f1_score(y_test, y_pred, average='weighted')
#f_score = f1_score(y_test_res, y_pred, average='weighted')
f_score

0.7906544567289585

10 sub-classifiers

In [82]:
n_samples = [0.1, 0.25]
n_features = [0.25, 0.5, 0.75]

samples_all = svd_x_train.shape[0]
samples_all_test = svd_x_test.shape[0]
features_all = svd_x_train.shape[1]

In [83]:
def get_minor_classifiers(samples_perc, features_perc):
    feature_list = [n for n in range(500)]
    classifiers = []
    
    for samples in samples_perc:
        for features in features_perc:
            f = sample(feature_list, int(features_all * features))
            x_train_f = svd_x_train[:,f]
            x_test_f = svd_x_test[:,f]
            
            x_train_s, y_train_s = resample(x_train_f, y_train, n_samples=int(samples * samples_all), replace=False, random_state=0)
            #x_test_s, y_test_s = resample(svd_x_test, y_test, n_samples=int(samples * samples_all_test), replace=False, random_state=0)
            
            svm_clf = SVC(probability=True)
            svm_clf.fit(x_train_s, y_train_s)
            
            y_pred = svm_clf.predict(x_test_f)
            pred = svm_clf.predict_proba(x_test_f)
            
            classifiers.append((y_pred, pred))
            
    return classifiers

In [84]:
clas = get_minor_classifiers(n_samples, n_features)

In [63]:
def average_pred(predictions):
    #predictions = list of tuples 
    m = len(predictions[0][0])
    all_results = [[0 for x in range(classes_count)] for y in range(m)] 
    results = [0] * m
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(classes_count):
                all_results[i][j] += pred_proba[i][j]
    for i in range(m):
        results[i] = all_results[i].index(max(all_results[i]))
    return results

def majority_pred(predictions):
    m = len(predictions[0][0])
    results = [0] * m
    for i in range(m):
        all_results = [0 for x in range(classes_count)] 
        for (pred, _) in predictions:
            all_results[pred[i]] += 1
        results[i] = all_results.index(max(all_results))        
    return results

def borda_pred(predictions):
    m = len(predictions[0][0])
    all_results = [[0 for x in range(classes_count)] for y in range(m)] 
    results = [0] * m
    
    def get_final_borda_points(predictions):
        return np.argsort(np.argsort(predictions)).tolist()

    for (_, pred_proba) in predictions:
        for i in range(m):
            pred_proba[i] = get_final_borda_points(pred_proba[i])
    for (_, pred_proba) in predictions:
        for i in range(m):
            for j in range(classes_count):
                all_results[i][j] += pred_proba[i][j]
    for i in range(m):
        results[i] = all_results[i].index(max(all_results[i]))
    return results

In [103]:
res = majority_pred(clas)

In [104]:
accuracy = accuracy_score(y_test, res)

In [105]:
accuracy

0.6768454593733404

In [55]:
def make_noise_label(labels, percent):
    labels_with_noise = deepcopy(labels)
    arr_size = labels.shape[0]
    indexes = [n for n in range(arr_size)]
    indexes_to_change = sample(indexes, int(arr_size * percent))
    
    for i in indexes_to_change:
        old_val = labels[i]
        new_val = randint(0, classes_count)
        while old_val == new_val:
            new_val = randint(0, classes_count)
        labels_with_noise[i] = new_val
        
    return labels_with_noise

In [114]:
clf_100iter = SVC(probability=True, max_iter=50) #221 dla 100, 24 dla 10, 46 dla 20, 112 dla 50

with elapsed_timer() as t:
    clf_100iter.fit(svd_x_train, y_train)

print("duration: " + str(t.duration))

duration: 111.54180649999944


C:\Users\majer\Anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Wyniki jakości klasyfikatorów:

In [107]:
UWAGA NIE PUSZCZAĆ BEZ POWODU - linijka niezakomentowana żeby uniknąć restartowania kernela
#cross-validation -> wybór metody fuzji decyzji?
scores = cross_validate(clf, svd_x_train, y_train, cv=5, scoring=('accuracy', 'neg_log_loss')) #zrobić klasę Model, robiącą model z 10 klasyfikatorów
print(scores['test_accuracy'])
print(scores['test_neg_log_loss'])

array([0.87538665, 0.87627044, 0.88201502, 0.87892179, 0.877542  ])

In [123]:
y_pred = clf.predict(svd_x_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy score: {0:0.2f}'.format(accuracy))

Accuracy score: 0.79


In [121]:
y_pred_proba = clf.predict_proba(svd_x_test)
loss = log_loss(y_test, y_pred_proba)
print('Loss score: {0:0.2f}'.format(loss))

Loss score: 0.72


In [129]:
average_precision = average_precision_score(y_test, y_pred, pos_label=1)
print('Precision-recall score: {0:0.2f}'.format(average_precision))

ValueError: multiclass format is not supported